In [1]:
import numpy as np
import keras.utils.np_utils as kutils
from sklearn.model_selection import train_test_split
import keras.callbacks as callbacks
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.utils import plot_model
from keras.optimizers import SGD

C:\Users\y884wang\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
apple = np.load('apple.npy')
potato = np.load('potato.npy')
basketball = np.load('basketball.npy')
watermelon = np.load('watermelon.npy')
circle = np.load('circle.npy')
compass = np.load('compass.npy')
donut = np.load('donut.npy')
soccerball = np.load('soccer ball.npy')
wheel = np.load('wheel.npy')
cookie = np.load('cookie.npy')

In [3]:
apple = np.c_[apple, np.zeros(apple.shape[0])]
potato = np.c_[potato, np.ones(potato.shape[0])]
basketball = np.c_[basketball, np.ones(basketball.shape[0])*2]
watermelon = np.c_[watermelon, np.ones(watermelon.shape[0])*3]
circle = np.c_[circle, np.ones(circle.shape[0])*4]
compass = np.c_[compass, np.ones(compass.shape[0])*5]
donut = np.c_[donut, np.ones(donut.shape[0])*6]
soccerball = np.c_[soccerball, np.ones(soccerball.shape[0])*7]
wheel = np.c_[wheel, np.ones(wheel.shape[0])*8]
cookie = np.c_[cookie, np.ones(cookie.shape[0])*9]

In [4]:
Data = np.concatenate((apple, potato,basketball,watermelon,circle,compass,donut,soccerball,wheel,cookie),axis=0)
del apple, potato, basketball, watermelon, circle, compass, donut, soccerball, wheel, cookie
X = Data[:,0:784]
Y = Data[:,784]
del Data

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [6]:
del X, Y

In [7]:
X_train = X_train.reshape(-1, 28,28, 1)

In [8]:
X_train.shape

(1220204, 28, 28, 1)

In [9]:
X_test = X_test.reshape(-1,28,28,1)

In [10]:
X_test.shape

(305051, 28, 28, 1)

In [11]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.
X_test = X_test / 255.

In [12]:
y_train = kutils.to_categorical(Y_train)
y_test = kutils.to_categorical(Y_test)

In [13]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=13)

In [14]:
X_valid.shape

(122021, 28, 28, 1)

In [15]:
y_train.shape

(1098183, 10)

In [16]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [17]:
batch_size = 128
epochs = 20
num_classes = 10

In [18]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(28,28,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(num_classes, activation='softmax'))

In [24]:
callbacks = [ModelCheckpoint('weights',
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                            mode='auto')
            ]
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["acc"])
model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs,
                   callbacks=callbacks,
                   validation_data=(X_valid, y_valid))

Train on 1098183 samples, validate on 122021 samples
Epoch 1/20
1098183/1098183 [==============================] - 1265s 1ms/step - loss: 0.4695 - acc: 0.8371 - val_loss: 0.3923 - val_acc: 0.8630

Epoch 00001: val_acc improved from -inf to 0.86300, saving model to weights
Epoch 2/20
1098183/1098183 [==============================] - 1264s 1ms/step - loss: 0.3699 - acc: 0.8716 - val_loss: 0.3684 - val_acc: 0.8722

Epoch 00002: val_acc improved from 0.86300 to 0.87218, saving model to weights
Epoch 3/20
1098183/1098183 [==============================] - 1257s 1ms/step - loss: 0.3448 - acc: 0.8800 - val_loss: 0.3581 - val_acc: 0.8746

Epoch 00003: val_acc improved from 0.87218 to 0.87458, saving model to weights
Epoch 4/20
1098183/1098183 [==============================] - 1255s 1ms/step - loss: 0.3283 - acc: 0.8853 - val_loss: 0.3547 - val_acc: 0.8767

Epoch 00004: val_acc improved from 0.87458 to 0.87670, saving model to weights
Epoch 5/20
1098183/1098183 [==============================

In [25]:
test_eval = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

Test loss: 0.4633625543305276
Test accuracy: 0.8679958433166348
